In [ ]:
from datetime import datetime
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.hooks.S3_hook import S3Hook

def upload_to_s3(filename: str, key: str, bucket_name: str) -> None:
    hook = S3Hook(aws_conn_id='s3_conn')
    hook.load_file(filename=filename, key=key, bucket_name=bucket_name)

default_args = {
    'owner': 'airflow',
    'start_date': datetime(2023, 11, 27),
    'depends_on_past': False,
    'retries': 1,
}

with DAG(
    dag_id='s3_dag',
    schedule_interval='@once',
    default_args=default_args,
    catchup=False,
) as dag:
    task_upload_to_s3 = PythonOperator(
        task_id='upload_to_s3',
        python_callable=upload_to_s3,
        op_kwargs={
            'filename': '/opt/airflow/student/student.csv',
            'key': 'student.csv',
            'bucket_name': 'tempcase',
        }
    )
